In [38]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import itertools
import toml


def wait_for_key(key=32):
    # while key != cv2.waitKey(30):
    #     pass
    cv2.waitKey(0)

# unpack the image under this path
# camera config will be stored in toml file under this path as well
path = 'chief/data'

In [33]:
def read_raw(path, size=(1080, 1440)):
    with open(path, 'r') as f:
        img = np.fromfile(f, dtype=np.uint8)
        img = img.reshape(size)
        return img

def read_images(paths):
    images = []
    for i, path in enumerate(paths):
        images.append(read_raw(path))
    return images

def generate_obj_points(grid):
    objp = np.zeros((grid[0] * grid[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:grid[0], 0:grid[1]].T.reshape(-1, 2)*20
    return objp

# can be useful for debugging
def show_img_3d(img):
    XX, YY = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    ax.plot_surface(YY, XX, img, cmap='jet')

def detect_corners(img, grid):
    """
    grid must match the number of the inner corners along width, height in img.
    It doesn't include corner on the bounardy of grid and is not the number of blocks, either. 
    """
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    mean, std = np.mean(img), np.std(img)
    img[img > mean + std] = mean + std
    ret, corners = cv2.findChessboardCorners(img, grid, None)
    if ret:
        term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 0.1)
        cv2.cornerSubPix(img, corners, (5, 5), (-1, -1), term)
    return ret, corners

def process_image(folder, grid=(16, 10), debug=False):
    objp = generate_obj_points(grid)
    objpoints = []
    imgpoints = []
    total, success = 0, 0
    if debug:
        debug_folder = os.path.join(folder, '.debug')
        os.makedirs(debug_folder, exist_ok=True)
    
    files = [file for file in os.listdir(folder) if not file.startswith('.')]
    if debug:
        files = files[:20]

    for img_file in tqdm(files):
        img_path = os.path.join(folder, img_file)
        img = read_raw(img_path)
        total += 1
        ret, corners = detect_corners(img, grid)
        if ret:
            success += 1
            objpoints.append(objp)
            imgpoints.append(corners)
            if debug:
                img_debug = img.copy()
                cv2.drawChessboardCorners(img_debug, grid, corners, ret)
                cv2.imwrite(os.path.join(debug_folder, img_file.rsplit(".", 1)[0]+'.png'), img_debug)

    print('Total: {}, Success: {}'.format(total, success))    
    return objpoints, imgpoints

In [39]:
grid = (16, 10)
objpoints, imgpoints = process_image(path + '/calibration_images', grid, debug=True)
rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, grid, None, None)
# save camera_matrix and dist_coefs with toml
with open(path + '/camera.toml', 'w') as f:
    toml.dump({'camera_matrix': camera_matrix.tolist(), 'dist_coefs': dist_coefs.tolist()}, f)
print(camera_matrix, dist_coefs)

100%|██████████| 20/20 [00:25<00:00,  1.27s/it]


Total: 20, Success: 8
[[2.51078918e+03 0.00000000e+00 2.71873280e+02]
 [0.00000000e+00 2.60398957e+03 4.91179390e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[-6.86290758e-01  1.69838913e-01 -6.97877543e-04  7.54176883e-02
   8.84974919e-01]]


In [42]:

test_img = read_raw(path + '/calibration_images/13_29_28_524.raw')
# load camera_matrix and dist_coefs with toml
with open(path + '/camera.toml', 'r') as f:
    data = toml.load(f)
camera_matrix = np.array(data['camera_matrix'])
dist_coefs = np.array(data['dist_coefs'])
print(camera_matrix, dist_coefs)


[[2.51078918e+03 0.00000000e+00 2.71873280e+02]
 [0.00000000e+00 2.60398957e+03 4.91179390e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[-6.86290758e-01  1.69838913e-01 -6.97877543e-04  7.54176883e-02
   8.84974919e-01]]


In [ ]:
mapx, mapy = cv2.initUndistortRectifyMap(camera_matrix, dist_coefs, None, cv2.getOptimalNewCameraMatrix(
    camera_matrix, dist_coefs, test_img.shape[::-1], 0)[0], test_img.shape[::-1], cv2.CV_32FC1)
img_undistorted = cv2.remap(
    test_img, mapx, mapy, cv2.INTER_LINEAR)
plt.figure()
plt.imshow(test_img, cmap='gray')
plt.figure()
plt.imshow(img_undistorted, cmap='gray')


In [27]:
a = os.listdir(path + '/calibration_images')
print(a)

['13_29_32_009.raw', '13_29_27_016.raw', '13_29_37_025.raw', '13_29_28_013.raw', '13_29_28_524.raw', '13_29_43_017.raw', '13_29_36_514.raw', '13_29_40_510.raw', '.debug', '13_29_29_012.raw', '13_29_48_012.raw', '13_29_35_516.raw', '13_29_33_008.raw', '13_29_44_525.raw', '13_29_37_513.raw', '13_29_51_518.raw', '13_29_42_019.raw', '13_29_24_020.raw', '13_29_34_516.raw', '13_29_45_525.raw', '13_29_39_510.raw', '13_29_26_526.raw', '13_29_24_508.raw', '13_29_41_508.raw', '13_29_36_025.raw', '13_29_52_007.raw', '13_29_29_523.raw', '13_29_26_015.raw', '13_29_53_516.raw', '13_29_48_520.raw', '13_29_40_022.raw', '13_29_32_518.raw', '13_29_47_522.raw', '13_29_25_019.raw', '13_29_46_013.raw', '13_29_46_524.raw', '13_29_33_519.raw', '13_29_45_016.raw', '13_29_38_022.raw', '13_29_47_013.raw', '13_29_34_007.raw', '13_29_31_010.raw', '13_29_25_528.raw', '13_29_31_520.raw', '13_29_53_027.raw', '13_29_50_520.raw', '13_29_44_016.raw', '13_29_51_009.raw', '13_29_50_010.raw', '13_29_39_022.raw', '13_29_42